<a href="https://colab.research.google.com/github/johyunkang/python_stat/blob/main/chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 10 통계적 추정

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/ch4_scores400.csv')
display(df.head(3))

scores = np.array(df['score'])
print('scores:', scores[:3])

p_mean = np.mean(scores) # 모평균
p_var = np.var(scores) # 모분산
print('모평균:', p_mean, ', 모분산:', p_var)

np.random.seed(0)
n=20
sample = np.random.choice(scores, n) # 20개의 랜덤샘플
print('20개 랜덤샘플:', sample)
s_mean = np.mean(sample)
print('20개 랜덤 표본평균:', s_mean)

n_samples = 10_000
samples = np.random.choice(scores, (n_samples, n)) # 20개의 랜덤샘플 1만개 생성
print('20개의 1만개 랜덤샘플 길이:', len(samples))

,score
0,76
1,55
2,80


scores: [76 55 80]
모평균: 69.53 , 모분산: 206.6691
20개 랜덤샘플: [49 60 72 86 52 61 77 91 80 56 69 67 90 56 75 79 60 79 68 81]
20개 랜덤 표본평균: 70.4
20개의 1만개 랜덤샘플 길이: 10000


## 10.1 점추정

### 10.1.2 모분산의 점추정

모분산의 불편추정량이 되는 표본통계량은 `불편분산 (unbiased variance)` 임

- 불편분산 : 표본분산에서 나누는 수 n을 n-1로 하는 다음식으로 계산

$s2 = \dfrac {1} {n-1} \displaystyle \sum_{i=1}^n(X_i - \bar{X})^2$

In [2]:
sample_u_vars = np.var(samples, axis=1, ddof=1) # 불편분산이라 ddof=1
print('1만개 표본분산의 평균:', np.mean(sample_u_vars))

print('샘플 100만개 표본분산:', np.var(np.random.choice(scores, 1_000_000), ddof=1))

1만개 표본분산의 평균: 207.20479631578948
샘플 100만개 표본분산: 207.2059178361167


## 10.2 구간추정

### 10.2.1 정규분포의 모평균 구간추정 : 모분산을 알고 있는 경우  
(정규분포 가정하고 계산)

모집단의 정규분포를 가정했으므로 표본평균 X_bar 는 아래 식을 따름

$\bar{X} = N(\mu, \dfrac{\sigma^2} {n})$

- 표준오차 (standard error) : 추정량의 표준편차

$표준오차 = \sqrt{\dfrac {\sigma^2}{n}}$

$Z= (\bar{X} - \mu) / \sqrt{\dfrac{\sigma^2}{n}} $

In [3]:
rv = stats.norm() # 정규분포
lcl = s_mean - rv.isf(0.025) * np.sqrt(p_var / n) # 95% 신뢰하한 (lower confidence limit)
ucl = s_mean - rv.isf(0.975) * np.sqrt(p_var / n) # 신뢰상한 (upper confidence limit)
print('95% 신뢰하한:', lcl, ', 95% 신뢰상한:', ucl) # 95% 신뢰구간
print('95% 신뢰구간:', lcl ,' ~ ', ucl)
# 해석은 100번 중 95번의 비율로 모평균이 신뢰구간에 포함된다.
# (모평균 실제 값은 하나 이기 때문)

95% 신뢰하한: 64.09956018535247 , 95% 신뢰상한: 76.70043981464754
95% 신뢰구간: 64.09956018535247  ~  76.70043981464754


### 10.2.2 정규분포의 모분산 구간추정 : 모평균을 모르는 경우

모집단의 정규분포를 가정하고, 모평균은 모르는 경우를 말함

**카이제곱분포**를 사용하며, 불편분산 s<sup>2</sup>에 

$Y = \dfrac {(n-1)s^2} {\sigma^2}$ 

이라는 변환을 함으로써 $Y \sim  \chi^2(n-1)$ 이 된다고 알려져 있음


> 모평균 &mu;가 알려져 있지 않을 때, 신뢰수준 100(1-&alpha;)%의 신뢰구간은

>$ \dfrac {(n-1)s^2} {χ_{a/2}^2(n-1)} , \dfrac {(n-1)s^2} {χ_{1-a/2}^2(n-1)}$

으로 추정됨

In [19]:
u_var = np.var(sample, ddof=1)
print('표본분산:', u_var)
rv = stats.chi2(df=n-1)
lcl = (n-1) * u_var / rv.isf(0.025) # Lower Confidence Limit
ucl = (n-1) * u_var / rv.isf(0.975) # Upper Confidence Limit

print('LCL:', lcl, ', UCL:', ucl)
print('모분산의 95% 신뢰구간은 [', lcl, ', ',  ucl, '] 이 되었음')
print(f'모분산은 {p_var} 였으므로 구간내에 포함되어 있다는 것도 확인할 수 있음')

표본분산: 158.25263157894742
LCL: 91.52471947132238 , UCL: 337.5955129124592
모분산의 95% 신뢰구간은 [ 91.52471947132238 ,  337.5955129124592 ] 이 되었음
모분산은 206.6691 였으므로 구간내에 포함되어 있다는 것도 확인할 수 있음


### 10.2.3 정규분포의 모평균 구간추정 : 모분산을 모르는 경우 

- 모분산을 모르는 상황에서 모평균의 구간추정에 관해 설명

- 모분산 &sigma;<sup>2</sup> 대신, 불편분산 s<sup>2</sup> 사용

$t = \dfrac {\bar{X} - \mu } {\sqrt{\dfrac{s^2}{n}}}$

- t는 자유도가 n-1 인 t 분포를 따름

In [21]:
rv = stats.t(df=n-1)
lcl = s_mean - rv.isf(0.025) * np.sqrt(u_var/ n)
ucl = s_mean - rv.isf(0.975) * np.sqrt(u_var/ n)
print(f'모평균의 95% 신뢰구간은 [{lcl} , {ucl}] 입니다')
print(f'모평균의 값은 {p_mean}점이었으므로, 구간내에 포함되어 있다는 것도 확인 ')

모평균의 95% 신뢰구간은 [64.51244886603887 , 76.28755113396114] 입니다
모평균의 값은 69.53점이었으므로, 구간내에 포함되어 있다는 것도 확인 


### 10.2.4 베르누이 분포의 모평균 구간추정 

정규분포가 아닌, 정권의 지지율이나 국민의 흡연율이라는 2진 변수의 모집단의 비율 p를 추정하고 싶은 상황 

In [24]:
PATH = '/content/drive/MyDrive/Colab Notebooks/누구나파이썬통계분석/data/'
enquete_df = pd.read_csv(PATH + 'ch10_enquete.csv')
display(enquete_df.head(3))
enquete = np.array(enquete_df['known'])
n = len(enquete)
print(f'n: {n}, sample 5개:{enquete[:5]}')

,known
0,1
1,0
2,1


n: 1000, sample 5개:[1 0 1 1 1]


Bern(p)의 기댓값이 p, 분산이 p(1-p) 이므로

표본평균 $\bar{X}$의 기댓값은 p, 분산은 $\dfrac {p(1-p)}{n}$가 됨

In [27]:
s_mean = enquete.mean()
print('표본평균:', s_mean)
rv = stats.norm()
lcl = s_mean - rv.isf(0.025) * np.sqrt(s_mean * (1 - s_mean) / n)
ucl = s_mean - rv.isf(0.975) * np.sqrt(s_mean * (1 - s_mean) / n)
print(f'상품A에 대한 인지도 p의 95% 신뢰구간은 [{lcl:.3}, {ucl:.3}] 입니다.')

표본평균: 0.709
상품A에 대한 인지도 p의 95% 신뢰구간은 [0.681, 0.737] 입니다.


### 10.2.5 포아송 분포의 모평균 신뢰구간 

- 모집단의 정규분포를 가정할 수 없음

- 샘플은 어떤 사이트에 대한 1시간당 액세스 수가 과거 72시간 분량이 들어있는 데이터를 사용하여, 이 사이트에 대한 1시간당 평균 액세스 수를 추정

- 단위 시간당 발생하는 건수에는 **포아송 분포**를 사용

- 각 데이터를 X<sub>1</sub>, X<sub>2</sub> 라고 하면, 이들은 서로 독립이고 Poi(&lambda;)를 따름

In [29]:
access_df = pd.read_csv(PATH + 'ch10_access.csv')
display(access_df.head(3))
n_access = np.array(access_df['access number'])
n = len(n_access)
print(f'n: {n}, sample 5개: {n_access[:5]}')

,access number
0,10
1,11
2,9


n: 72, sample 5개: [10 11  9  9 18]


In [35]:
s_mean = n_access.mean()
print('표본평균:', s_mean) # 1시간당 평균 10.44건 액세스

# 포아송 분포의 표본평균이 따르는 확률분포는 알지 못하므로
# 중심극한정리를 사용
rv = stats.norm()
lcl = s_mean - rv.isf(0.025) * np.sqrt(s_mean / n)
ucl = s_mean - rv.isf(0.975) * np.sqrt(s_mean / n)
print(f'사이트에 대한 평균 액세스 수의 95% 신뢰구간은 [{lcl}, {ucl}] 임')

표본평균: 10.444444444444445
사이트에 대한 평균 액세스 수의 95% 신뢰구간은 [9.697953549598202, 11.190935339290688] 임
